In [2]:
import os
import pickle
import zlib

import numpy as np
import torch
import psycopg2

from db.DocumentRepository import DocumentRepository
from ranker.Ranker import Ranker


In [3]:
os.system("""
    docker compose down;
    docker compose up -d --build db;
    sleep 15;
    """)

 Container project_mse-db-1  Stopping
 Container project_mse-db-1  Stopped
 Container project_mse-db-1  Removing
 Container project_mse-db-1  Removed
 Network project_mse_default  Removing
 Network project_mse_default  Removed


#0 building with "desktop-linux" instance using docker driver

#1 [db internal] load build definition from Dockerfile
#1 transferring dockerfile: 122B done
#1 DONE 0.0s

#2 [db internal] load metadata for docker.io/library/postgres:latest
#2 DONE 0.0s

#3 [db internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [db 1/2] FROM docker.io/library/postgres:latest
#4 DONE 0.0s

#5 [db internal] load build context
#5 transferring context: 32B done
#5 DONE 0.0s

#6 [db 2/2] COPY dump.sql /docker-entrypoint-initdb.d/
#6 CACHED

#7 [db] exporting to image
#7 exporting layers done
#7 writing image sha256:67c1644c2918172ee852e247262ca7123c812f5566eeb6a063940a17662af7dd done
#7 naming to docker.io/library/project_mse-db done
#7 DONE 0.0s


 Network project_mse_default  Creating
 Network project_mse_default  Created
 Container project_mse-db-1  Creating
 Container project_mse-db-1  Created
 Container project_mse-db-1  Starting
 Container project_mse-db-1  Started


0

In [3]:
ranker = Ranker()
documentRepository = ranker.documentRepository
tokenizer = documentRepository.tokenizer
all_docs = documentRepository.loadAllDocuments()

SC: Connected to the db. Now you can go and build the best search engine around!
Loading Documents first...
All documents loaded.
Now we load TF and IDF
TF and IDF loaded.


In [9]:
# takes long
enc_texts = [doc.enc_text for doc in all_docs]
max_length = max(len(doc) for doc in enc_texts)
padded_docs = [torch.cat([doc, torch.tensor([-1] * (max_length - doc.size(0)))]) for doc in enc_texts]
tensor_docs = torch.stack(padded_docs)
tensor_docs = tensor_docs.to(torch.int)
idf = ranker._compute_idfv2(tensor_docs)

tf, doc_lengths, max_term = ranker._calculate_tf_and_lengthsv2(tensor_docs)
avg_doc_len = sum(doc_lengths) / len(doc_lengths)
print("Works")

Works


1. Step: Save idf dictionary

In [15]:
# 1. Step: save idf
conn = documentRepository.connection
cur = documentRepository.cursor

for key, value in idf.items():
    cur.execute("INSERT INTO idfs (key, value) VALUES (%s, %s) ON CONFLICT (key) DO UPDATE SET value = %s", (key, value, value))

conn.commit()
print("IDF saved! :)")

IDF saved! :)


In [16]:
# 2. Step: load idf again
cur.execute("SELECT key, value FROM idfs")
rows = cur.fetchall()

# convert
loaded_dict = {key: value for key, value in rows}

print(loaded_dict)

{999: 0.8210520802698537, 1004: 0.27584576355458257, 1006: 0.13202640003388588, 1007: 0.1323855954400467, 1010: 0.05349041414382498, 1011: 0.03295243767008476, 1012: 0.020668462290645968, 1013: 0.2526866180383306, 1037: 0.09362788204971567, 1040: 0.8296728233137606, 1045: 0.6163901407651048, 1059: 1.8412737349922406, 1075: 0.2750167479630616, 1516: 0.8203370283981279, 1996: 0.046377655681256555, 1997: 0.0640036524036279, 1998: 0.052494893902323224, 1999: 0.06098857474265006, 2000: 0.038987852777197955, 2006: 0.1768291930459286, 2007: 0.250864939566742, 2012: 0.17065113351900033, 2013: 0.25674671293762896, 2015: 0.07989847469769451, 2017: 0.3549904020164344, 2019: 0.19158541993684258, 2024: 0.31884719954776997, 2035: 0.15307598869749295, 2038: 1.2329757684174298, 2039: 1.1865539315661806, 2041: 0.9301550854652609, 2055: 0.1622875497794887, 2057: 0.3660477187815675, 2062: 0.5221669031814979, 2063: 0.2624586827721232, 2078: 0.35118258019070625, 2097: 0.9111896809857496, 2099: 0.4053864329

In [44]:
documentRepository = DocumentRepository()
conn = documentRepository.connection
cur = documentRepository.cursor

SC: Connected to the db. Now you can go and build the best search engine around!


In [49]:
# 3. Step: Save tf
print(type(tf))

n = tf.size(dim=0)
m = tf.size(dim=1)

# serialize tf tensor
tf_numpy = tf.numpy()
serialized_data = pickle.dumps(tf_numpy)
compressed_data = zlib.compress(serialized_data)

# Chunk the compressed data
chunk_size = 1024 * 1024  # 1 MB
chunks = [compressed_data[i:i + chunk_size] for i in range(0, len(compressed_data), chunk_size)]

# Insert data into the table
for chunk_id, chunk in enumerate(chunks):
    cur.execute(
        'INSERT INTO tfs (chunk_id, data) VALUES (%s, %s)',
        (chunk_id, psycopg2.Binary(chunk))
    )
conn.commit()
print("Inserted tf :)")

<class 'torch.Tensor'>
Inserted tf, doc_lengths, max_term, avg_doc_len :)


In [50]:
cur.execute('SELECT data FROM tfs ORDER BY chunk_id')
chunks = cur.fetchall()
compressed_data = b''.join([chunk[0] for chunk in chunks])

# Decompress and deserialize the data
serialized_data = zlib.decompress(compressed_data)
numpy_array_restored = pickle.loads(serialized_data)

# Convert the NumPy array back to a PyTorch tensor
tensor_restored = torch.tensor(numpy_array_restored, dtype=torch.int32)
print(tensor_restored.shape)

torch.Size([6355, 30511])


In [45]:
# Step 5: Save tf metadata
print(type(doc_lengths), type(max_term), type(avg_doc_len))

cur.execute(
    'INSERT INTO tf_meta (doc_lengths, max_term, avg_doc_len) VALUES (%s, %s, %s)',
    (doc_lengths, max_term, avg_doc_len)
)

conn.commit()
print("Inserted tf, doc_lengths, max_term, avg_doc_len :)")

<class 'list'> <class 'int'> <class 'float'>
Inserted tf, doc_lengths, max_term, avg_doc_len :)


In [46]:
# 6: Reconstruct tf metadata

cur.execute('SELECT doc_lengths, max_term, avg_doc_len FROM tf_meta WHERE id = 1')  # Assuming you want the first row
meta_data = cur.fetchone()
doc_lengths = meta_data[0]
max_term = meta_data[1]
avg_doc_len = meta_data[2]
print(doc_lengths)
print(max_term)
print(avg_doc_len)

[Decimal('491'), Decimal('2626'), Decimal('289'), Decimal('2340'), Decimal('525'), Decimal('2344'), Decimal('3368'), Decimal('3330'), Decimal('2194'), Decimal('987'), Decimal('1398'), Decimal('345'), Decimal('2317'), Decimal('6544'), Decimal('4870'), Decimal('1366'), Decimal('315'), Decimal('2688'), Decimal('2503'), Decimal('2088'), Decimal('2113'), Decimal('664'), Decimal('2212'), Decimal('257'), Decimal('2043'), Decimal('2045'), Decimal('2174'), Decimal('548'), Decimal('264'), Decimal('551'), Decimal('1913'), Decimal('2200'), Decimal('1486'), Decimal('995'), Decimal('273'), Decimal('334'), Decimal('2714'), Decimal('268'), Decimal('2206'), Decimal('3793'), Decimal('260'), Decimal('1023'), Decimal('2132'), Decimal('1381'), Decimal('2618'), Decimal('1961'), Decimal('1543'), Decimal('1655'), Decimal('1595'), Decimal('1229'), Decimal('1350'), Decimal('412'), Decimal('1261'), Decimal('690'), Decimal('1930'), Decimal('685'), Decimal('1282'), Decimal('1678'), Decimal('1044'), Decimal('1369')

Nice, we were able to compute tf and idf offline just based on our index. That is necessary to make BM25 faster.

In [51]:
documentRepository.overwrite_dump()

SC: Successfully overwritten the old dump. Now you only need to push it to the repository!
